In [11]:
# coding: utf-8
import pandas as pd
from gensim.models import *
import numpy as np
import time
from ast import literal_eval
import lstm
import random
import pickle

# TODO1: 文件输入en，jp的txt文件的时候，需要检查1. 是否行数一直 2. 是否有\n\n的问题出现
# 目前是手动删除对应的\n\n行[已完成]

# TODO1: 每次都要计算imilarity table效率太低了。1000行还可接受，但是更多行就不行了
# 首先边计算边存储到一个dictionary去[已完成]

epoch = 40
maxlen = 0 # Default: 0 -> infinite
k = 10
wnl = 0
dim = 200
nan = np.empty(dim)
counter = 0 
start_time = time.time()
dic_mapping = {}
# np.random.seed(1234)
random.seed(1234)

dir_cluster_center = './data/cluster-skmeans/'
model_name_en = "./data/model-en/W2Vmodle.bin"
model_name_jp = "./data/model-jp/W2Vmodle.bin"
dir_mapping = "./data/mapping/mapping_en_" + str(k) + ".csv"

log_filename1 = "./log/output_unmatch_jp.log"
output_unmatch = open(log_filename1,'w')

if False:
	# # For ALL
	sample_size = ""
	dir_txt_en = "./data/news/wo_empty_line_en" + sample_size + ".txt"
	dir_txt_jp = "./data/news/wo_empty_line_jp" + sample_size + ".txt"

else:
	# #For sample
	# #sample size
	sample_size = "_1000"
	dir_txt_en = "./data/news/wo_empty_line_en" + sample_size + ".txt"
	dir_txt_jp = "./data/news/wo_empty_line_jp" + sample_size + ".txt"



# Call mapping_word
def mapping_article(article,model):
	# start_time = time.time()
	global counter
	print counter
	counter = counter + 1
	tokens = article.split()
	tokens_mapping=[mapping_word(word,model) for word in tokens]
	# print "DEBUG: Finish 1 line-------------"
	# print time.time() - start_time
	return tokens_mapping

# Call: get_vector
# Find the nearest en-cluster for a given word
def mapping_word(word,model):
	# 1. get the center for each en-cluster
	df_center_en=find_cluster_center(dir_cluster_center,'en')
	
	if word in dic_mapping:
		cluster_number = dic_mapping[word]
		# print "IN!"
		return cluster_number
	else:
		# 2. find the word2vec expression
		vec=get_vector(word,model)
		# print "OUT----"
		# print "DEBUG: ",vec
		
		if np.all(vec!=nan): 
			# 3. calculate the similarity matrix
			similarity_matrix_en = \
				np.array(df_center_en).dot(vec) # ????? Have a check!
			# print "DEBUG, similarity_matrix_en ="
			# print "with shape of ", np.shape(similarity_matrix_en)
			
			# 4. Get the maximum one that can present this cluster
			cluster_number=similarity_matrix_en.argmax()+1

			# 5. add the mapping to the dictionary
			dic_mapping[word] = cluster_number

			return cluster_number
		else:
			print "Error: vec == NaN"
			return None


def get_vector(word,model):
	word=word.rstrip() # remove all '\n' and '\r'
	# word=word.lower()
	# baseform=getVector.getBase(word,wnl)
	# print "DEBUG: ",model['good']
	# print "DEBUG: baseform= ", baseform
	try:
		vecW=model[word] #!!!Maybe the word is not existed
	except Exception,e:
		# info=''
		# counter_NaN+=1 #increase 1 to NaN counter
		# info+=repr(e)+"\n" #create log information
		# logout.write(info) #write log information to log file				
		#new 3.15: generate a useless list for deleting in the next stage
		output_unmatch.write(word) # no \n is needed since the 
		output_unmatch.write('\n')
		print "---Warning: Word ["+word+"] Vector Not Found ---"
		return nan
	else:
		# vecW=getVector.vecNorm(vecW) #Normalized the raw vector
		# print "(the new length of the vector is:",LA.norm(vecW),")"
		# info+=baseform+": OK!\n" #create log information
		# logout.write(info) #write log information to log file
		# fout.write(rawVoc) #add in 16/3/17
		# good_list.append(rawVoc)
		#append the new vector to the matrix
		#if the vector is the first element in the matrix: 'good_vecs', reshape it
		return vecW

# Find the center of each cluster
def find_cluster_center(cluster_centroid_dir,lang_name):
	cluster_center_filename=cluster_centroid_dir + "centroid_" + lang_name + str(k) + ".csv"
	df_cluster_center = pd.read_csv(cluster_center_filename,index_col=0)
	# print "DEBUG: df_cluster_center is [" +  cluster_center_filename + "]" 
	# print df_cluster_center
	return df_cluster_center


def map_to_jp_vector(vector_en, df_mapping):
	result = []
	for cluster_name_en in vector_en:
		if cluster_name_en != None:
			cluster_name_converted = df_mapping.iloc[cluster_name_en-1].mapping_parsed
			result += list(cluster_name_converted)
		else:
			result = None
	# print result
	return result


def evaluate_1(k):
	xa = df_train_1['xa'][k]
	return evaluate(xa)

def evaluate(xa):
	xa_0_result = df_train_1['xb'].apply(sls.predict_similarity,args=(xa,))
	ranking = xa_0_result.rank(ascending = False)[k]
	# print ranking
	return ranking


def evaluate_all(df):
	df_result = df.xa.apply(evaluate)
	return df_result


# -----------------------Prepare the mapped Data----------------

def prepare_trainig(dir_en, dir_jp):

	# Read the saved mapping results:
	df_en_mapping = pd.read_csv(dir_en)
	df_jp_mapping = pd.read_csv(dir_jp)

	print "Reading english Data:", len(df_en_mapping)
	print "Reading english Data:", len(df_jp_mapping)

	sample_size = len(df_en_mapping)

	assert len(df_en_mapping) == len(df_jp_mapping)

	# Convert mapping to list type and then concat to the a list
	print "Merging the English and Japanes news dataframe"
	df_train_1 = pd.concat([df_en_mapping, df_jp_mapping], axis = 1)
	df_train_1['transformation_en'] = df_train_1.transformation_en.apply(literal_eval)
	df_train_1['transformation_jp'] = df_train_1.transformation_jp.apply(literal_eval)
	# df_train_1['similarity'] = pd.Series(np.ones(int(sample_size[1:]),))
	df_train_1['similarity'] = pd.Series(np.ones(sample_size,)*5)
	df_train_1['dis_similarity'] = pd.Series(np.ones(sample_size,)*1)



	# prepare the multi-lingual cluster mapping
	print "Read mapping file and convert it to tuple from string"
	df_mapping = pd.read_csv(dir_mapping)
	df_mapping['mapping_parsed'] = df_mapping.mapping.map(lambda x: literal_eval(x)) 

	# Call map_to_jp_vector()
	print "Mapping English clusters to Japanese clusters"
	df_train_1['en2jp_projection'] = \
		df_train_1['transformation_en'].apply(map_to_jp_vector,args=(df_mapping,))

	# Remove null line
	print "Drop the null line"
	df_train_1 = df_train_1.dropna(subset=['en2jp_projection'])

	# Convert list of cluster number to a string 
	print "Convert cluster names(list) to cluster namse(string)"
	df_train_1[['xa','xb']] = df_train_1[['transformation_en','en2jp_projection']].applymap(lambda x:' '.join(str(v) for v in x))


	# Expand the training data
	xb_wrong = df_train_1.xb.iloc[random.sample(xrange(len(df_train_1)),len(df_train_1))]
	xb_wrong.index = df_train_1.index
	print (xb_wrong == df_train_1.xb).value_counts()
	df_train_1['xb_wrong'] = xb_wrong


	# Convert dateframe to list
	train_1 = df_train_1[['xa','xb','similarity']].values.tolist()
	train_2 = df_train_1[['xa','xb_wrong','dis_similarity']].values.tolist()

	return train_1, train_2, df_train_1

# # Called by find_ranking
# # Given 2 list of projection results, calculate there L1-norm similarity
# def cal_similarity(a, b):
# 	diff = np.linalg.norm(a - b, 1, axis=1)
# 	sim = np.exp(-diff)
# 	# len(diff)
# 	return sim

# # Find the ranking results with respect to real pairs
# def find_ranking(projection1, projection2):
# 	sim_results = []
# 	rank_results = []
# 	for i, proj1 in enumerate(projection1):
# 		sim = cal_similarity(proj1, projection2)
# 		rank = pd.Series(sim).rank(ascending = False)[i]
# 		sim_results.append(sim)
# 		rank_results.append(rank)
# 	return sim_results, rank_results


In [12]:
	model_en = Word2Vec.load(model_name_en)
	model_jp = Word2Vec.load(model_name_jp)

	# -----------------------Mapping Raw News Data----------------
	# Read news data
	df_en = pd.read_table(dir_txt_en, names=["en_article"])
	df_jp = pd.read_table(dir_txt_jp, names=["jp_article"])

	# Mapping cluster name For Enlgish news
	# and save the file 
	if False:
		print "Mapping cluster name For Enlgish news"
		start_time = time.time()
		df_en['transformation_en'] = \
			df_en.en_article.apply(mapping_article,args=(model_en,))
		df_en.to_csv("./data/mapping/en_mapped_" + str(k) + sample_size + ".csv",index=False)
		print time.time() - start_time

	# Mapping cluster name For Japanese news
	# and save the file
	if False:
		print "Mapping cluster name For Japanese news"
		start_time = time.time()
		df_jp['transformation_jp'] = \
			df_jp.jp_article.apply(mapping_article,args=(model_jp,))
		df_jp.to_csv("./data/mapping/jp_mapped_" + str(k) + sample_size + ".csv",index=False)
		print time.time() - start_time

	# -----------------Formatting the data------------------------

	if False:
		# Prepare For the training data
		dir_en = "./data/mapping/en_mapped_"+str(k) + sample_size + ".csv"
		dir_jp = "./data/mapping/jp_mapped_" + str(k) + sample_size + ".csv"
		train_1, train_2, df_train_1 = prepare_trainig(dir_en, dir_jp)

		# Prepare For the testing data
		sample_size = "_1k2k"
		dir_en = "./data/mapping/en_mapped_"+str(k) + sample_size + ".csv"
		dir_jp = "./data/mapping/jp_mapped_" + str(k) + sample_size + ".csv"
		test_1, test_2, df_test_1 = prepare_trainig(dir_en, dir_jp)

		# ----save the prepared data into pickle-----------------------
		root_dir = "pickles/"
		with open(root_dir + "train_1.p", 'wb') as handle:
		    pickle.dump(train_1, handle)
		with open(root_dir + "train_2.p", 'wb') as handle:
		    pickle.dump(train_2, handle)
		with open(root_dir + "test_1.p", 'wb') as handle:
		    pickle.dump(test_1, handle)
		with open(root_dir + "test_2.p", 'wb') as handle:
		    pickle.dump(test_2, handle)
		with open(root_dir + "df_train_1.p", 'wb') as handle:
		    pickle.dump(df_train_1, handle)
		with open(root_dir + "df_test_1.p", 'wb') as handle:
		    pickle.dump(df_test_1, handle)

	else:
		root_dir = "pickles/"
		# ------load the exited prepared data from pickle---------------
		train_1 = pickle.load(open(root_dir + "train_1.p",'rb'))
		train_2 = pickle.load(open(root_dir + "train_2.p",'rb'))
		test_1 = pickle.load(open(root_dir + "test_1.p",'rb'))
		# test_2 = pickle.load(open(root_dir + "test_2.p",'rb')) 
		df_train_1 = pickle.load(open(root_dir + "df_train_1.p",'rb'))
		df_test_1 = pickle.load(open(root_dir + "df_test_1.p",'rb'))


In [13]:
train = train_1 + train_2

In [132]:
test_1[2]

['7 7 5 7 6 6 5 5 7 2 9 7 7 5 3 2 7 5 4 2 5 2 4 5 5 4 3 2 7 7 10 7 7 4 8 8 3 5 7 2 8 6 7 2 7 5 5 5 5 7 7 5 3 2 3 3 3 3 3 3 6 5 5 3 5 3 5 5 8 10 7 5 5 2 9 10 4 4 4 4 4 4 4 4 4 5 8 4 2 10 2 4 6 4',
 '10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 8 2 3 10 3 10 3 1 5 8 2 10 3 1 10 8 2 1 8 2 10 1 1 10 5 8 2 10 3 10 3 4 10 3 10 3 10 4 4 5 1 10 3 8 2 4 9 7 2 10 3 8 2 10 3 1 1 1 1 10 3 10 3 1 5 8 2 5 5 5 5 5 5 9 7 2 1 1 5 1 5 1 1 4 4 10 3 1 1 8 2 3 4 10 10 10 10 10 10 10 10 10 1 4 10 8 2 4 8 2 10 9 7 2 10',
 5.0]

In [131]:
df_test_1

,en_article,transformation_en,jp_article,transformation_jp,similarity,dis_similarity,en2jp_projection,xa,xb,xb_wrong,pred_ranks
0,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,1 9 7 2 1 1 10 3 1 8 2 10 3 8 2 10 3 1 8 2 6 1...,2.0
1,european central bank keep interest rate stead...,"[7, 7, 5, 5, 6, 6, 5, 5, 5, 3, 2, 5, 5, 6, 4, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 1, 9, 7, 2, 9, 7, 2, 1, 1, 1...",7 7 5 5 6 6 5 5 5 3 2 5 5 6 4 5 5 4 5 5 4 3 2 ...,10 3 10 3 1 1 9 7 2 9 7 2 1 1 1 5 8 2 1 1 9 7 ...,1 1 4 4 4 6 10 3 10 3 8 2 8 2 6 1 4 1 1 3 4 1 ...,12.0
2,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 2, 9, 7, 7, 5, 3, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 2 9 7 7 5 3 2 7 5 4 2 5 2 4 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 8 2 3 10...,1 5 8 2 8 2 1 9 7 2 9 7 2 1 1 1 9 7 2 9 7 2 1 ...,2.0
3,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 1 1 8 2 1 4 4...,2.0
4,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 5, 5, 2, 5, 8, 8, 10,...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 5 5 2 5 8 8 10 7 7 7 7 7 7 7 5...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 1 1 8 2 1 4 4...,8 2 6 10 3 4 6 8 2 9 7 2 10 8 2 8 2 10 3 4 8 2...,1.5
5,european central bank hold borrow rate thursda...,"[7, 7, 5, 7, 6, 6, 5, 2, 1, 2, 1, 3, 5, 4, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 2 1 2 1 3 5 4 2 5 5 4 7 5 9 7 7 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 8 2 6 8 2 6 5 1...,8 2 8 2 9 7 2 6 9 7 2 1 1 10 3 1 8 2 1 1 6 6 6...,2.0
6,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,9 7 2 8 2 4 8 2 4 10 3 8 2 6 10 3 6 4 8 2 1 1 ...,2.0
7,china annual consumer inflation slow more aggr...,"[2, 6, 5, 3, 4, 5, 5, 6, 5, 5, 5, 5, 5, 5, 7, ...",中国国家統計局 発表 する 12月 消費者物価指数 前年 上昇 する 11月 上昇 鈍化 低...,"[8, 10, 7, 4, 8, 9, 2, 7, 8, 2, 4, 4, 1, 7, 7,...",5.0,1.0,"[8, 2, 9, 7, 2, 1, 5, 10, 1, 1, 9, 7, 2, 1, 1,...",2 6 5 3 4 5 5 6 5 5 5 5 5 5 7 4 4 1 6 7 5 2 1 ...,8 2 9 7 2 1 5 10 1 1 9 7 2 1 1 1 1 1 1 10 3 10...,9 7 2 1 9 7 2 1 1 9 7 2 9 7 2 9 7 2 1 1 8 2 10...,1.0
8,european central bank unexpected cut interest ...,"[7, 7, 5, 5, 1, 6, 6, 2, 5, 5, 5, 7, 5, 3, 5, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 1, 6, 9, 7, 2, 9, 7, 2, 8, 2...",7 7 5 5 1 6 6 2 5 5 5 7 5 3 5 5 5 5 5 4 2 7 7 ...,10 3 10 3 1 1 6 9 7 2 9 7 2 8 2 1 1 1 10 3 1 5...,1 1 10 10 3 6 9 7 2 6 1 10 9 7 2 1 1 1 10 3 10...,3.0
9,european central bank hold borrow rate thursda...,"[7, 7, 5, 7, 6, 6, 5, 2, 1, 2, 1, 3, 5

In [15]:
df_train_1

,en_article,transformation_en,jp_article,transformation_jp,similarity,dis_similarity,en2jp_projection,xa,xb,xb_wrong
0,u.s. commerce department seasonal adjust annua...,"[5, 2, 9, 6, 6, 6, 6, 2, 6, 6]",米 商務省 発表 する 建設 支出 以下 とおり 季節調整 年率 総 建設 支出 民間 部門...,"[2, 3, 10, 7, 3, 10, 8, 8, 9, 8, 9, 3, 10, 7, ...",5.0,1.0,"[1, 8, 2, 3, 9, 7, 2, 9, 7, 2, 9, 7, 2, 9, 7, ...",5 2 9 6 6 6 6 2 6 6,1 8 2 3 9 7 2 9 7 2 9 7 2 9 7 2 8 2 9 7 2 9 7 2,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...
1,chinese budget smartphone maker xiaomi plan se...,"[2, 6, 2, 2, 2, 1, 2, 2, 5, 6, 2, 2, 2, 2, 10,...",中国 低価格 スマホ メーカー 小米科技 () 雷軍 董事長 以上 販売 する 計画 明らか...,"[7, 5, 8, 2, 8, 3, 8, 7, 3, 8, 7, 10, 7, 7, 9,...",5.0,1.0,"[8, 2, 9, 7, 2, 8, 2, 8, 2, 8, 2, 6, 8, 2, 8, ...",2 6 2 2 2 1 2 2 5 6 2 2 2 2 10 8 5 6 1 8 2 5 2...,8 2 9 7 2 8 2 8 2 8 2 6 8 2 8 2 1 9 7 2 8 2 8 ...,10 3 10 3 6 10 3 10 8 2 6 6 10 9 7 2 1 10 10 1...
2,samsung electronics co ltd fell more percent t...,"[2, 2, 2, 2, 5, 5, 5, 5, 2, 5, 6, 5, 5, 5, 6, ...",ソウル 株式市場 サムスン電子 株価 超 下落 する 安値 付ける 一部 アナリスト ウォン...,"[3, 3, 3, 9, 3, 2, 7, 3, 3, 9, 9, 3, 8, 6, 8, ...",5.0,1.0,"[8, 2, 8, 2, 8, 2, 8, 2, 1, 1, 1, 1, 8, 2, 1, ...",2 2 2 2 5 5 5 5 2 5 6 5 5 5 6 5 6 1 6 5 5 5 5 ...,8 2 8 2 8 2 8 2 1 1 1 1 8 2 1 9 7 2 1 1 1 9 7 ...,1 1 1 1 1 4 6 6 8 2 4 10 10 10 8 2 10 1 1 4 4 ...
3,fiat deal gain full control chrysler group llc...,"[2, 1, 5, 1, 8, 2, 8, 10, 5, 2, 1, 5, 5, 5, 5,...",欧州 株式市場 イタリア 自動車 大手 FIAT 株価 一時 急伸 する 8月 以来 高値 ...,"[10, 3, 10, 9, 4, 6, 9, 2, 8, 7, 4, 3, 3, 3, 7...",5.0,1.0,"[8, 2, 6, 1, 6, 4, 8, 2, 4, 4, 1, 8, 2, 6, 1, ...",2 1 5 1 8 2 8 10 5 2 1 5 5 5 5 7 7 2 7 2 1 1 5...,8 2 6 1 6 4 8 2 4 4 1 8 2 6 1 1 1 1 10 3 10 3 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 8 2 3 10...
4,fiat share jump thursday it strike deal gain f...,"[2, 1, 5, 5, 2, 8, 1, 5, 1, 8, 2, 8, 10, 7, 4,...",欧州 株式市場 イタリア 自動車 大手 FIAT 株価 一時 急伸 する 8月 以来 高値 ...,"[10, 3, 10, 9, 4, 6, 9, 2, 8, 7, 4, 3, 3, 3, 7...",5.0,1.0,"[8, 2, 6, 1, 1, 8, 2, 4, 6, 1, 6, 4, 8, 2, 4, ...",2 1 5 5 2 8 1 5 1 8 2 8 10 7 4 7 2 7 2 1 1 5 8...,8 2 6 1 1 8 2 4 6 1 6 4 8 2 4 4 10 3 10 10 3 8...,10 3 10 3 1 6 9 7 2 9 7 2 1 1 1 6 5 9 7 2 8 2 ...
5,u.s. treasuries price rise thursday benchmark ...,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 6, 10,...",カッコ 前 営業日 米 東部 時間 時分 前 営業日 終盤 米 東部 時間 時分 前 営業日...,"[10, 4, 3, 2, 5, 3, 4, 4, 3, 3, 2, 5, 3, 4, 4,...",5.0,1.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 7, 2, 1, 1, 1, ...",5 5 5 5 5 5 5 5 5 6 5 5 5 6 10 5 5 5 2 7 2 6 5...,1 1 1 1 1 1 1 1 1 9 7 2 1 1 1 9 7 2 4 1 1 1 8 ...,10 3 1 1 1 1 1 1 1 10 3 10 10 4 4 8 2 6 4 4 1 ...
6,auction detail see 3-year note,"[1, 1, 5, 1, 5]",米財務省 銘柄 統合 入札 実施 する 発表 する 発行 いずれ 償還 1月 11月 11月,"[6, 9, 7, 6, 4, 7, 10, 7, 6, 10, 6, 9, 8, 8]",5.0,1.0,"[6, 6, 1, 6, 1]",1 1 5 1 5,6 6 1 6 1,9 7 2 10 3 6 1 1 1 1 1 5 5 9 7 2 10 3 8 2 8 2 ...
7,brent crude sink barrel thursday libya prepare...,"[5, 5, 5, 5, 5, 8, 1, 1, 2, 8, 4, 4, 4, 2, 4, ...",原油 先物 相場 リビア 大 規模 油田 生産 停止 要因 なる いる 抗議活動 休止 する...,"[9, 3, 1, 9, 2, 9, 7, 4, 4, 9, 7, 4, 4, 4, 7, ...",5.0,1.0,"[1, 1, 1, 1, 1, 4, 6, 6, 8, 2, 4, 10, 10, 10, ...",5 5 5 5 5 8 1 1 2 8 4 4 4 2 4 5 5 8 8 5 2 2 1 ...,1 1 1 1 1 4 6 6 8 2 4 10 10 10 8 2 10 1 1 4 4 ...,1 8 2 1 1 1 8 2 1 1 1 1 1 1 9 7 2 1 1 1 1 1 10...
8,gauge u.s. factory activity hold 2-1/2-year hi...,"[5, 5, 4, 4, 7, 5, 5, 5, 2, 8, 2, 2, 9, 5, 3, ...",米 労働省 発表 する 12月 週 新規 失業保険 申請 件数 季節調整 前週 り 連続 減...,"[2, 8, 10, 7, 4, 9, 8, 8, 8, 8, 9, 3, 3, 8, 8,...",5.0,1.0,"[1, 1, 10, 10, 10, 3, 1, 1, 1, 8, 2, 4, 8, 2, ...",5 5 4 4 7 5 5 5 2 8 2 2 9 5 3 5 4 5 5 5 5 10 4...,1 1 10 10 10 3 1 1 1 8 2 4 8 2 8 2 3 1 5 1 10 ...,8 2 8 2 8 2 1 9 7 2 8 2 9 7 2 6 1 9 7 2 9 7 2 ...
9,work massive panama canal extension project ma...,"[8, 1, 8, 2, 1, 1, 7, 2, 1, 9, 5, 2, 9, 6, 9, ...",パナマ運河 拡張 工事 めぐる 建設 会社 成る 企業 連合 パナマ運河 PC コスト 超過...,"[9, 3, 9, 9, 3, 9, 4, 6, 7, 9, 2, 9, 9, 3, 5, ...",5.0,1.0,"[4, 6, 4, 8, 2, 6, 6, 10, 3, 8, 2, 6, 3, 1, 8,...",8 1 8 2 1 1 7 2 1 9 5 2 9 6 9 6 7 2 8 5 6 9 2 ...,4 6 4 8 2 6 6 10 3 8 2 6 3 1 8 2 3 9 7 2 3 9 7...

In [16]:
df_test_1

,en_article,transformation_en,jp_article,transformation_jp,similarity,dis_similarity,en2jp_projection,xa,xb,xb_wrong
0,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,1 9 7 2 1 1 10 3 1 8 2 10 3 8 2 10 3 1 8 2 6 1...
1,european central bank keep interest rate stead...,"[7, 7, 5, 5, 6, 6, 5, 5, 5, 3, 2, 5, 5, 6, 4, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 1, 9, 7, 2, 9, 7, 2, 1, 1, 1...",7 7 5 5 6 6 5 5 5 3 2 5 5 6 4 5 5 4 5 5 4 3 2 ...,10 3 10 3 1 1 9 7 2 9 7 2 1 1 1 5 8 2 1 1 9 7 ...,1 1 4 4 4 6 10 3 10 3 8 2 8 2 6 1 4 1 1 3 4 1 ...
2,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 2, 9, 7, 7, 5, 3, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 2 9 7 7 5 3 2 7 5 4 2 5 2 4 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 8 2 3 10...,1 5 8 2 8 2 1 9 7 2 9 7 2 1 1 1 9 7 2 9 7 2 1 ...
3,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 1 1 8 2 1 4 4...
4,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 5, 5, 2, 5, 8, 8, 10,...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 5 5 2 5 8 8 10 7 7 7 7 7 7 7 5...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 1 1 8 2 1 4 4...,8 2 6 10 3 4 6 8 2 9 7 2 10 8 2 8 2 10 3 4 8 2...
5,european central bank hold borrow rate thursda...,"[7, 7, 5, 7, 6, 6, 5, 2, 1, 2, 1, 3, 5, 4, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 2 1 2 1 3 5 4 2 5 5 4 7 5 9 7 7 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 8 2 6 8 2 6 5 1...,8 2 8 2 9 7 2 6 9 7 2 1 1 10 3 1 8 2 1 1 6 6 6...
6,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,9 7 2 8 2 4 8 2 4 10 3 8 2 6 10 3 6 4 8 2 1 1 ...
7,china annual consumer inflation slow more aggr...,"[2, 6, 5, 3, 4, 5, 5, 6, 5, 5, 5, 5, 5, 5, 7, ...",中国国家統計局 発表 する 12月 消費者物価指数 前年 上昇 する 11月 上昇 鈍化 低...,"[8, 10, 7, 4, 8, 9, 2, 7, 8, 2, 4, 4, 1, 7, 7,...",5.0,1.0,"[8, 2, 9, 7, 2, 1, 5, 10, 1, 1, 9, 7, 2, 1, 1,...",2 6 5 3 4 5 5 6 5 5 5 5 5 5 7 4 4 1 6 7 5 2 1 ...,8 2 9 7 2 1 5 10 1 1 9 7 2 1 1 1 1 1 1 10 3 10...,9 7 2 1 9 7 2 1 1 9 7 2 9 7 2 9 7 2 1 1 8 2 10...
8,european central bank unexpected cut interest ...,"[7, 7, 5, 5, 1, 6, 6, 2, 5, 5, 5, 7, 5, 3, 5, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 1, 6, 9, 7, 2, 9, 7, 2, 8, 2...",7 7 5 5 1 6 6 2 5 5 5 7 5 3 5 5 5 5 5 4 2 7 7 ...,10 3 10 3 1 1 6 9 7 2 9 7 2 8 2 1 1 1 10 3 1 5...,1 1 10 10 3 6 9 7 2 6 1 10 9 7 2 1 1 1 10 3 10...
9,european central bank hold borrow rate thursda...,"[7, 7, 5, 7, 6, 6, 5, 2, 1, 2, 1, 3, 5, 4, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利

In [17]:
	train = train_1 + train_2

	# True to training the data, False to laod the existed data
	print "Now the maxlen =", maxlen
	if False:
		dir_file = "weights/2017012218009_e40_1k1k_l0.p"
		print "Starting to training the model..., saving to", dir_file
		sls=lstm.LSTM(dir_file, maxlen, load=False, training=True)
		sls.train_lstm(train, epoch, train_1, test_1)
		sls.save_model()
	else:
		dir_file = "weights/2017012218009_e40_1k1k_l0.p"
		print "NO Training. Load the existed model:", dir_file
		sls=lstm.LSTM(dir_file, maxlen, load=True, training=False)


Now the maxlen = 0
NO Training. Load the existed model: weights/2017012218009_e40_1k1k_l0.p


In [24]:
sim_train = sls.evaluate2(train_1,[1, 5, 10])

In [25]:
sim_test = sls.evaluate2(test_1,[1, 5, 10])

In [44]:
sim_train[0]

999


In [ ]:
sim_test[1]

In [37]:
pd.Series(sim_train[0]).describe()
# 这个数据和 2017-1-15（1）的数据吻合

count    999.000000
mean       2.008008
std        2.085938
min        1.000000
25%        1.000000
50%        1.500000
75%        2.000000
max       27.000000
dtype: float64

In [39]:
pd.Series(sim_test[0]).describe()
# 这个数据和 2017-1-15（1）的数据吻合

count    1000.000000
mean        3.608000
std         4.799309
min         1.000000
25%         1.000000
50%         2.000000
75%         3.500000
max        28.500000
dtype: float64

In [74]:
df_train_1['pred_ranks'] = sim_train[0]
df_test_1['pred_ranks'] = sim_test[0]

In [48]:
df_train_1

,en_article,transformation_en,jp_article,transformation_jp,similarity,dis_similarity,en2jp_projection,xa,xb,xb_wrong,pred_ranks
0,u.s. commerce department seasonal adjust annua...,"[5, 2, 9, 6, 6, 6, 6, 2, 6, 6]",米 商務省 発表 する 建設 支出 以下 とおり 季節調整 年率 総 建設 支出 民間 部門...,"[2, 3, 10, 7, 3, 10, 8, 8, 9, 8, 9, 3, 10, 7, ...",5.0,1.0,"[1, 8, 2, 3, 9, 7, 2, 9, 7, 2, 9, 7, 2, 9, 7, ...",5 2 9 6 6 6 6 2 6 6,1 8 2 3 9 7 2 9 7 2 9 7 2 9 7 2 8 2 9 7 2 9 7 2,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,5.0
1,chinese budget smartphone maker xiaomi plan se...,"[2, 6, 2, 2, 2, 1, 2, 2, 5, 6, 2, 2, 2, 2, 10,...",中国 低価格 スマホ メーカー 小米科技 () 雷軍 董事長 以上 販売 する 計画 明らか...,"[7, 5, 8, 2, 8, 3, 8, 7, 3, 8, 7, 10, 7, 7, 9,...",5.0,1.0,"[8, 2, 9, 7, 2, 8, 2, 8, 2, 8, 2, 6, 8, 2, 8, ...",2 6 2 2 2 1 2 2 5 6 2 2 2 2 10 8 5 6 1 8 2 5 2...,8 2 9 7 2 8 2 8 2 8 2 6 8 2 8 2 1 9 7 2 8 2 8 ...,10 3 10 3 6 10 3 10 8 2 6 6 10 9 7 2 1 10 10 1...,1.0
2,samsung electronics co ltd fell more percent t...,"[2, 2, 2, 2, 5, 5, 5, 5, 2, 5, 6, 5, 5, 5, 6, ...",ソウル 株式市場 サムスン電子 株価 超 下落 する 安値 付ける 一部 アナリスト ウォン...,"[3, 3, 3, 9, 3, 2, 7, 3, 3, 9, 9, 3, 8, 6, 8, ...",5.0,1.0,"[8, 2, 8, 2, 8, 2, 8, 2, 1, 1, 1, 1, 8, 2, 1, ...",2 2 2 2 5 5 5 5 2 5 6 5 5 5 6 5 6 1 6 5 5 5 5 ...,8 2 8 2 8 2 8 2 1 1 1 1 8 2 1 9 7 2 1 1 1 9 7 ...,1 1 1 1 1 4 6 6 8 2 4 10 10 10 8 2 10 1 1 4 4 ...,1.0
3,fiat deal gain full control chrysler group llc...,"[2, 1, 5, 1, 8, 2, 8, 10, 5, 2, 1, 5, 5, 5, 5,...",欧州 株式市場 イタリア 自動車 大手 FIAT 株価 一時 急伸 する 8月 以来 高値 ...,"[10, 3, 10, 9, 4, 6, 9, 2, 8, 7, 4, 3, 3, 3, 7...",5.0,1.0,"[8, 2, 6, 1, 6, 4, 8, 2, 4, 4, 1, 8, 2, 6, 1, ...",2 1 5 1 8 2 8 10 5 2 1 5 5 5 5 7 7 2 7 2 1 1 5...,8 2 6 1 6 4 8 2 4 4 1 8 2 6 1 1 1 1 10 3 10 3 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 8 2 3 10...,1.5
4,fiat share jump thursday it strike deal gain f...,"[2, 1, 5, 5, 2, 8, 1, 5, 1, 8, 2, 8, 10, 7, 4,...",欧州 株式市場 イタリア 自動車 大手 FIAT 株価 一時 急伸 する 8月 以来 高値 ...,"[10, 3, 10, 9, 4, 6, 9, 2, 8, 7, 4, 3, 3, 3, 7...",5.0,1.0,"[8, 2, 6, 1, 1, 8, 2, 4, 6, 1, 6, 4, 8, 2, 4, ...",2 1 5 5 2 8 1 5 1 8 2 8 10 7 4 7 2 7 2 1 1 5 8...,8 2 6 1 1 8 2 4 6 1 6 4 8 2 4 4 10 3 10 10 3 8...,10 3 10 3 1 6 9 7 2 9 7 2 1 1 1 6 5 9 7 2 8 2 ...,3.5
5,u.s. treasuries price rise thursday benchmark ...,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 6, 10,...",カッコ 前 営業日 米 東部 時間 時分 前 営業日 終盤 米 東部 時間 時分 前 営業日...,"[10, 4, 3, 2, 5, 3, 4, 4, 3, 3, 2, 5, 3, 4, 4,...",5.0,1.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 7, 2, 1, 1, 1, ...",5 5 5 5 5 5 5 5 5 6 5 5 5 6 10 5 5 5 2 7 2 6 5...,1 1 1 1 1 1 1 1 1 9 7 2 1 1 1 9 7 2 4 1 1 1 8 ...,10 3 1 1 1 1 1 1 1 10 3 10 10 4 4 8 2 6 4 4 1 ...,1.0
6,auction detail see 3-year note,"[1, 1, 5, 1, 5]",米財務省 銘柄 統合 入札 実施 する 発表 する 発行 いずれ 償還 1月 11月 11月,"[6, 9, 7, 6, 4, 7, 10, 7, 6, 10, 6, 9, 8, 8]",5.0,1.0,"[6, 6, 1, 6, 1]",1 1 5 1 5,6 6 1 6 1,9 7 2 10 3 6 1 1 1 1 1 5 5 9 7 2 10 3 8 2 8 2 ...,1.0
7,brent crude sink barrel thursday libya prepare...,"[5, 5, 5, 5, 5, 8, 1, 1, 2, 8, 4, 4, 4, 2, 4, ...",原油 先物 相場 リビア 大 規模 油田 生産 停止 要因 なる いる 抗議活動 休止 する...,"[9, 3, 1, 9, 2, 9, 7, 4, 4, 9, 7, 4, 4, 4, 7, ...",5.0,1.0,"[1, 1, 1, 1, 1, 4, 6, 6, 8, 2, 4, 10, 10, 10, ...",5 5 5 5 5 8 1 1 2 8 4 4 4 2 4 5 5 8 8 5 2 2 1 ...,1 1 1 1 1 4 6 6 8 2 4 10 10 10 8 2 10 1 1 4 4 ...,1 8 2 1 1 1 8 2 1 1 1 1 1 1 9 7 2 1 1 1 1 1 10...,1.0
8,gauge u.s. factory activity hold 2-1/2-year hi...,"[5, 5, 4, 4, 7, 5, 5, 5, 2, 8, 2, 2, 9, 5, 3, ...",米 労働省 発表 する 12月 週 新規 失業保険 申請 件数 季節調整 前週 り 連続 減...,"[2, 8, 10, 7, 4, 9, 8, 8, 8, 8, 9, 3, 3, 8, 8,...",5.0,1.0,"[1, 1, 10, 10, 10, 3, 1, 1, 1, 8, 2, 4, 8, 2, ...",5 5 4 4 7 5 5 5 2 8 2 2 9 5 3 5 4 5 5 5 5 10 4...,1 1 10 10 10 3 1 1 1 8 2 4 8 2 8 2 3 1 5 1 10 ...,8 2 8 2 8 2 1 9 7 2 8 2 9 7 2 6 1 9 7 2 9 7 2 ...,7.0
9,work massive panama canal extension project ma...,"[8, 1, 8, 2, 1, 1, 7, 2, 1, 9, 5, 2, 9, 6, 9, ...",パナマ運河 拡張 工事 めぐる 建設 会社 成る 企業 連合 パナマ運河 PC コスト 超過...,"[9, 3, 9, 9, 3, 9, 4, 6, 7, 9, 2, 9, 9, 3, 5, ...",5.0,1.0,"[4, 6, 4, 8, 2, 6, 6, 10, 3, 8, 2, 6, 3, 1, 8,...",8 1 8 2 1 1 7 2 1 9 5 2 9 6 9 6 7 2 8 5 6 9 2 ...,4 

In [75]:
df_test_1

,en_article,transformation_en,jp_article,transformation_jp,similarity,dis_similarity,en2jp_projection,xa,xb,xb_wrong,pred_ranks
0,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,1 9 7 2 1 1 10 3 1 8 2 10 3 8 2 10 3 1 8 2 6 1...,2.0
1,european central bank keep interest rate stead...,"[7, 7, 5, 5, 6, 6, 5, 5, 5, 3, 2, 5, 5, 6, 4, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 1, 9, 7, 2, 9, 7, 2, 1, 1, 1...",7 7 5 5 6 6 5 5 5 3 2 5 5 6 4 5 5 4 5 5 4 3 2 ...,10 3 10 3 1 1 9 7 2 9 7 2 1 1 1 5 8 2 1 1 9 7 ...,1 1 4 4 4 6 10 3 10 3 8 2 8 2 6 1 4 1 1 3 4 1 ...,12.0
2,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 2, 9, 7, 7, 5, 3, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 2 9 7 7 5 3 2 7 5 4 2 5 2 4 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 8 2 3 10...,1 5 8 2 8 2 1 9 7 2 9 7 2 1 1 1 9 7 2 9 7 2 1 ...,2.0
3,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 1 1 8 2 1 4 4...,2.0
4,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 5, 5, 2, 5, 8, 8, 10,...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 5 5 2 5 8 8 10 7 7 7 7 7 7 7 5...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 1 1 8 2 1 4 4...,8 2 6 10 3 4 6 8 2 9 7 2 10 8 2 8 2 10 3 4 8 2...,1.5
5,european central bank hold borrow rate thursda...,"[7, 7, 5, 7, 6, 6, 5, 2, 1, 2, 1, 3, 5, 4, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 2 1 2 1 3 5 4 2 5 5 4 7 5 9 7 7 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 8 2 6 8 2 6 5 1...,8 2 8 2 9 7 2 6 9 7 2 1 1 10 3 1 8 2 1 1 6 6 6...,2.0
6,european central bank leave interest rate unch...,"[7, 7, 5, 7, 6, 6, 5, 5, 7, 1, 1, 1, 2, 2, 2, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 10, 3, 9, 7, 2, 9, 7, 2, 1, ...",7 7 5 7 6 6 5 5 7 1 1 1 2 2 2 4 2 5 4 5 7 7 2 ...,10 3 10 3 1 10 3 9 7 2 9 7 2 1 1 10 3 6 6 6 8 ...,9 7 2 8 2 4 8 2 4 10 3 8 2 6 10 3 6 4 8 2 1 1 ...,2.0
7,china annual consumer inflation slow more aggr...,"[2, 6, 5, 3, 4, 5, 5, 6, 5, 5, 5, 5, 5, 5, 7, ...",中国国家統計局 発表 する 12月 消費者物価指数 前年 上昇 する 11月 上昇 鈍化 低...,"[8, 10, 7, 4, 8, 9, 2, 7, 8, 2, 4, 4, 1, 7, 7,...",5.0,1.0,"[8, 2, 9, 7, 2, 1, 5, 10, 1, 1, 9, 7, 2, 1, 1,...",2 6 5 3 4 5 5 6 5 5 5 5 5 5 7 4 4 1 6 7 5 2 1 ...,8 2 9 7 2 1 5 10 1 1 9 7 2 1 1 1 1 1 1 10 3 10...,9 7 2 1 9 7 2 1 1 9 7 2 9 7 2 9 7 2 1 1 8 2 10...,1.0
8,european central bank unexpected cut interest ...,"[7, 7, 5, 5, 1, 6, 6, 2, 5, 5, 5, 7, 5, 3, 5, ...",欧州中央銀行 EC 主要政策金利 据え置く 上限 金利 限界 貸出金利 下限 金利 中銀 預...,"[7, 10, 9, 10, 10, 9, 8, 8, 8, 9, 10, 8, 9, 9,...",5.0,1.0,"[10, 3, 10, 3, 1, 1, 6, 9, 7, 2, 9, 7, 2, 8, 2...",7 7 5 5 1 6 6 2 5 5 5 7 5 3 5 5 5 5 5 4 2 7 7 ...,10 3 10 3 1 1 6 9 7 2 9 7 2 8 2 1 1 1 10 3 1 5...,1 1 10 10 3 6 9 7 2 6 1 10 9 7 2 1 1 1 10 3 10...,3.0
9,european central bank hold borrow rate thursda...,"[7, 7, 5, 7, 6, 6, 5, 2, 1, 2, 1, 3, 5

In [76]:
df_train_top1 = df_train_1[df_train_1['pred_ranks'] == 1]
df_test_top1 = df_test_1[df_test_1['pred_ranks'] == 1]

In [79]:
df_train_top5 = df_train_1[df_train_1['pred_ranks'] == 5]
df_test_top5 = df_test_1[df_test_1['pred_ranks'] == 5]

In [64]:
print(df_train_top1.iloc[0]['en_article'])
print(df_train_top1.iloc[0]['jp_article'])

chinese budget smartphone maker xiaomi plan sell handset more double number it sell it chairman say thursday reinforce company ambition outsell more expensive offering apple inc samsung electronics co ltd. lei jun who be also co-founder tech firm make projection sina weibo china most widely use microblogging site it be reposted xiaomi website we again promise we will least phone he write china be world big market smartphones lei say private hold xiaomi have sell smartphones percent increase sale revenue include tax rise percent yuan xiaomi sale growth far exceed projection global smartphone market which be see expand annual rate percent year accord research firm canalys xiaomi cheap yet sleek phone be popular china where samsung electronics remain market leader xiaomi handset sell much low price tag least expensive iphone model samsung galaxy note ii which can retail xiaomi even manage briefly rank sixth market share second quarter notch apple apple be poise boost it sale china it sign

In [65]:
print(df_train_top1.iloc[1]['en_article'])
print(df_train_top1.iloc[1]['jp_article'])

samsung electronics co ltd fell more percent thursday it low month weigh down analyst forecast low quarterly earnings due strong won currency weak margin it display business share fell equal much percent thursday won low late august stock close down percent it fifth consecutive session decline downtrend that have wipe nearly market value asia most valuable company samsung world large maker smartphones memory chip television be schedule report it october-december quarter earnings estimate tuesday company be forecast report record won operating profit october-december quarter percent increase previous quarter accord average estimate analyst thomson reuters i/b/e/s starmine smartestimate average most accurate analyst estimate forecast percent quarter-on-quarter drop samsung operating profit won we expect samsung performance be won low market consensus won big reason would be won-dollar exchange rate follow shrink margin organic light-emitting diode oled sale say lee seung-woo tech analyst

In [68]:
print(df_train_top1.iloc[4]['en_article'])
print(df_train_top1.iloc[4]['jp_article'])

brent crude sink barrel thursday libya prepare restart major oilfield slow economic expansion china world oil consumer libya national oil corp noc plan restart el sharara oilfield hop resume output day protester agree suspend strike that block field end october adding pressure oil growth factory activity china world second large fuel consumer slow late accord purchase manager index publish government hsbc weigh shrink export order brent crude fell cent tuesday barrel gmt u.s. crude be off cent market be shut wednesday new year oil market appear be run steam say christopher bellew broke jefferies bache production libya iran iraq united state will be close watch year addition sign further stimulus tapering u.s. federal reserve libya oil output be still less barrel day bpd down bpd july port eastern part country remain shut restart el sharara could see export start recover information libya need be take caution confirm fact could bring temporary increase libya production barrel day say ol

In [93]:
print(df_train_top5.iloc[1]['en_article'])
print(df_train_top5.iloc[1]['jp_article'])
print(df_train_top5.iloc[1])

singapore economy contract more expect fourth quarter manufacturing activity weaken data show thursday cast doubt market expectation slight pick-up growth accord advance estimate singapore ministry trade industry gdp contract annualised seasonal adjust percent final quarter july-september period reverse percent expansion third quarter be weak median percent contraction forecast reuters survey i think weakness be manufacturing side hope manufacturing recovery be finally show more visible it look it sputter again say chua hak bin head emerge asia economics bank america merrill lynch overall story be growth be still largely lead more steady service sector he add manufacturing sector contract annualised rate percent fourth quarter previous quarter year-on-year manufacturing growth moderate percent compare percent third quarter service sector contract annualised percent quarter-on-quarter grow percent year-on-year put perspective it follow very strong second quarter positive third quarter s

In [117]:
projection1_train, projection2_train = sls.seq2vec(train_1)
# projection1_test, projection2_test = sls.seq2vec(test_1)

# Calculate the rankings for this data set
sim_results_train, rank_results_train = find_ranking(projection1_train, projection2_train)

In [109]:
sim_results_train[63][63]

0.787112

In [120]:
# 【注意】比较分数时永远以日语文章为基准
# 找出63号日语文档对应的跨语言文章相似度分数
# 取前5个，发现弟
sim_results_train[63].argsort()[-5:][::-1]

array([371, 492, 297, 788,  63])

In [126]:
print(sim_results_train[63][371])
print(df_train_1.iloc[371]['en_article'])
print(df_train_1.iloc[371]['jp_article'])
print(sim_results_train[63][492])
print(df_train_1.iloc[492]['en_article'])
print(df_train_1.iloc[492]['jp_article'])
print(sim_results_train[63][297])
print(df_train_1.iloc[297]['en_article'])
print(df_train_1.iloc[297]['jp_article'])
print(sim_results_train[63][788])
print(df_train_1.iloc[788]['en_article'])
print(df_train_1.iloc[788]['jp_article'])
print(sim_results_train[63][63])
print(df_train_1.iloc[63]['en_article'])
print(df_train_1.iloc[63]['jp_article'])

0.864382
britain top equity index edge high late monday financial stock outpace weak mining share that fell new sign economic weakness china world top metal consumer rsa insurance rise percent sunday telegraph newspaper say probe it irish business be likely conclude account problem be isolated incident more writedowns would be need other insurer bank generally see cyclical stock be also demand expectation improve economic outlook europe united state would prove be positive sector insurer legal general gain percent bank barclays hsbc rise percent percent respective uk banking index be percent strong top sectoral gainer banking sector be clear expose improvement economic condition we be see uk sector be show good earnings momentum equal well say robert parkes equity strategist hsbc security blue-chip ftse index be up percent point gmt however gain be cap sharp decline mine stock uk mining index fell percent fresh sign economic weakness china survey show growth chinese service sector acti

NameError: name 'rank_results_test' is not defined